In [1]:
# expand the cell width to 100% of the content
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Aufgabenstellung

Wir haben uns für Thema 2 - Chaos entschieden.  
Es umfasst folgende sechs Unterthemen:
- Logistische Gleichung
- Feigenbaum-Diagramm
- Sensitivität
- Julia-Mengen
- Mandelbrotmenge
- Apfelmännchen und Selbstähnlichkeit

Zu jedem Thema soll eine Erklärung, eine Art kurzer Wiki-Eintrag verfasst werden, die dem Leser einen groben Überblick über das Thema vermittelt.  
Außerdem sollen die Sachverhalte wo möglich durch entsprechende Visualisierungen veranschaulicht werden. Dazu stellt Sage bereits einige Methoden zur Verfügung, insbesondere zur Darstellung von Julia- und Mandelbrotmengen. Da die Aufgabenstellung aber eigene Implementationen fordert, nutzen wir diese nicht sondern haben selbst exemplarische Lösungen gefunden.


# Implementierung

## Logistische Gleichung

Die logistische Gleichung ist eine relativ einfache, nichtlineare Gleichung, die sich im Gegensatz zu den uns bisher bekannten linearen Gleichungen für bestimmte Parameter scheinbar unvorhersehbar und chaotisch verhält.  
Sie ist rekursiv definiert durch die folgende Gleichung:  
<p style="text-align: center; font-size: 20px;">
<b> x<sub>n+1</sub> = r * x<sub>n</sub> * (1 - x<sub>n</sub>) </b> </p>  
    
### Populationsmodellierung durch exponentielles Wachstum
Die logistische Gleichung wurde 1845 von Pierre-François Verhulst, einem belgischen Mathematiker, zur Modellierung von Populationen entwickelt.  
Der damals übliche Ansatz dafür war, die Population durch eine exponentiell wachsende Funktion x<sub>n+1</sub> = r * x<sub>n</sub> zu modellieren.  
x<sub>n</sub> ist dabei die Größe der Population zum Zeitpunkt n, r eine Spezies-spezifische Wachstumsrate (i.d.R. > 1, sonst würde die Spezies einfach aussterben).  

Dies lässt sich zu Beginn so auch in der Realität beobachten, beispielsweise bei der Vermehrung von Zellen oder dem Wachstum von Pilzen.  
Eine Zelle teilt sich, man bekommt nach einem Zeitintervall zwei, nach zwei Intervallen vier, nach drei Intervallen acht Zellen und so weiter. In diesem Beispiel wäre also r = 2.
Auch für Tiere und Menschen scheint dies intuitiv: Je dichter ein gegebener Lebensraum mit einer Spezies besiedelt ist, desto höher ist auch die Wahrscheinlichkeit, dass sich zwei paarungswillige und -fähige Individuen treffen und sich vermehren.  

Allerdings ist auch offensichtlich, dass diese Modellierung nicht die gesamte Entwicklung beschreiben kann:  
Die Funktion wächst exponentiell und wird somit irgendwann sehr schnell unendlich groß, wohingegen die Population einer Spezies in der Realität begrenzt ist. Irgendwann reichen beispielsweise die vorhandenen Ressourcen im Lebensraum (Platz, Nahrung) nicht mehr für alle aus, wodurch die Sterblichkeit steigt und so die effektive Wachstumsrate (Geburtenrate - Sterberate) abfällt. Die Population wächst dann langsamer oder schrumpft sogar. Diese äußeren Einflüsse fasst man in der *Kapazität K* eines Lebensraums zusammen. Sie beschreibt die maximale Größe einer Population in diesem Lebensraum.

### Ergänzung durch wachstumshemmende Terme
Verhulst erkannte dieses Problem und löste es, indem er neben dem wachstumstreibenden Term x<sub>n</sub> noch einen wachstumshemmenden in die Gleichung einbaute.  
Beschreibt die Kapazität K wie oben definiert die maximale Individuenanzahl, so muss die Population umso langsamer wachsen, je näher sich ihre Größe schon an die Kapazität angenähert hat.  
Dies erreicht man zum Beispiel durch das Hinzufügen des Faktors (K - x<sub>n</sub>). Nun liefert diese Differenz aber Faktoren im Intervall [0, K], also auch solche > 1, die das Wachstum weiter steigern statt es zu hemmen. Man teilt also nochmal durch K, um als Bildbereich das Intervall [0, 1] zu erhalten: (1 - x<sub>n</sub> / K). Dadurch entsteht die logistische Gleichung x<sub>n+1</sub> = r * x<sub>n</sub> * (1 - x<sub>n</sub> / K).  
K wird in der Regel einfach weggelassen beziehungsweise K = 1 gesetzt.  
Auch die nachfolgende Simulation betrachtet die Gleichung wie zu Beginn notiert ohne Betrachtung einer spezifischen Kapazität.
In ihr erkennt man gut, dass sich die Populationsgröße für 1 < r < 3 fast immer irgendwann stabilisiert.  
Für r <= 1 stirbt die Population aus, für r >= 3 schwingt die Gleichung jedoch oder wird sogar - wie eingangs erwähnt - scheinbar unvorhersehbar chaotisch.


In [82]:
var('r', 'x')
log_equation(r, x) = r * x * (1 - x)
    
@interact
def plot_le(x0 = slider(0, 1, default=0.5, step_size=0.1, label="Startwert x<sub>0</sub>"), 
            r  = slider(0, 4, default=3.8, step_size=0.1, label="Wachstum r"),
            darstellung = selector(['Punkte', 'Kurve'], label="Darstellung", default='Kurve')):
    """Plottet interaktiv die logistische Gleichung als Kurve oder diskrete Punkte"""
    x = x0
    p = [(0, x)]
    for i in range(1, 100):
        x = log_equation(r, x)
        p += [(i, x)]
    result = line(p) if darstellung == 'Kurve' else points(p)

    result.show(figsize=10, ymin=0, ymax=1, axes_labels=['$n$','$X$'])

Interactive function <function plot_le at 0x6fdc7231ae8> with 3 widgets
  x0: TransformFloatSlider(value=0.5, …

## Feigenbaum Diagramm

Das Feigenbaumdiagramm ist ein nach dem US-amerikanischen Physiker Mitchell Feigenbaum benanntes Diagramm, welches nichtlineare Systeme, wie z.B. die eben beschriebene logistische Gleichung, visualisiert.

Im Folgenden sieht man eine Implementierung für das Feigenbaumdiagramm der logistischen Gleichung. <br>
Zur Berechnung werden die spezifizierte Nummer an Punkten mit zufälligen x- und r-Werten im Gesammten Bereich generiert. Danach wird die logistische Gleichung N-mal auf jeden Punkt andewandt und die Punkte anschließend in der Form (r,x<sub>N</sub>) geplottet. <br>
Über die Regler lassen sich einige Werte, wie die Anzahl der Punkte, Iterationen pro Punkt oder der betrachtete R-Bereich individuell einstellen.<br>
Den R<sub>Min</sub>-Wert größer als R<sub>Max</sub> zu wählen ist logisch unsinnig und erzeugt demnach kein Diagramm.

Der Focus<sub>Points</sub> Wert gibt an, welcher Anteil der zufällig generierten Punkte in dem Segment ab 2,5 liegen (z.B. Focus<sub>Points</sub> = 2 --> 1/3 der Punkte zwischen 0 und 2,5, 2/3 zwischen 2,5 und 4). Ist der Wert 0, so werden alle Punkte gleichwertig generiert. 
Um kleinere Änderungen Vorzunehmen können sie einen Schieber eines Reglers selektieren und mit den Pfeiltasten anpassen.
<br>Um größere Änderungen Vorzunehmen klicken sie bitte auf den Regler an der gewünschten Stelle, anstatt den Schieber zu verschieben.
<br>Bei einer hohen Zahl an Punkten oder Iterationen kann es etwas dauern, bis das eingestellte Diagramm erstellt wurde.

In [3]:
@interact
def plot_fig(NumPoints = slider(1, 100000, default=500, step_size=1, label="Num<sub>Points</sub>"),
             focus = slider(1, 5, default=0, step_size=1, label="Focus<sub>Points</sub>"),
             N = slider(0, 100, default=5, step_size=1, label="N<sub>Iterations</sub>"),
             rMin = slider(0.00, 4, default=0, step_size=0.1, label="R<sub>Min</sub>"),
             dotSize = slider(1, 50, default=1, step_size=1, label="Size<sub>Dot</sub>"),
             auto_update = false):
    
    p = []
    rMax = 4
    for i in range(0, NumPoints):
        if focus != 0:
            if (i < (NumPoints / (focus + 1)) * focus):
                p += [(uniform(2.5,rMax),random())]
            else:
                p += [(uniform(0,2.5),random())]
        else:
            p += [(uniform(0,rMax),random())]
    #p = list(filter(lambda x: x[0] > rMin, p))
    for i in range(len(p)):
        for n in range(N):
            # Aus performance-technischen Gründen wird hier direkt die Formel und nicht die Funktion von oben benutzt
            # Refactored: Verwendung einer Sage Funktion statt einer Klasse für log_equation, jetzt einheitliche Nutzung der log_equation
            
            # Punkt = (r, xN)
            p[i] = (p[i][0], log_equation(p[i][0], p[i][1])) 
            # p[i] = (p[i][0], p[i][0] * p[i][1] * (1 - p[i][1]))
            
    #print(len(ps))
    show(points(p, size=dotSize), axes_labels=['$R-Value$','$X-Value$'], figsize=8, ymin=0, ymax=1, xmin=rMin, xmax=rMax)

Manual interactive function <function plot_fig at 0x6fe1c20ce18> with 5 widgets
  NumPoints: TransformIntSlide…

### Feigenbaum-Konstante
Mit höheren Iterationsstufen ist hier nun das klassische Feigenbaum-Verhalten immer besser zu erkennen. Die X-Werte der einzelnen Punkte streben schlussendlich immer zu einem (bei höherem R zu mehreren) Fixpunkt. Die Kurve dieser Fixpunkte liegt bei R-Werten unter 1,0 bei 0 (logisch: Wachstum unter 1,0 --> Schrumpfen). Die nun folgende Kurve spaltet sich ab einem gewissen R-Wert auf und beginnt zwischen beiden Teilen zu osziliieren. Jeder dieser Teile spaltet sich nun wieder und wieder und wieder... Diese Punkte der Abspaltung werden als Bifurkationspunkte bezeichnet. Der Graph wirkt zunächst noch recht geordnet, allerdings verkürzt sich die Spanne zwischen den Bifurkationspunkten immer weiter (immer ca. 4,66 mal kurzer als das Vorgängerintervall), bis die Kurve schlussendlich wieder Chaotisch wird.

Feigenbaum hat hierzu nun nachgewiesen, dass in diesem nun chaotischen Verhalten doch wieder Ordnung zu finden ist.
1978 veröffentlichte er die sog. (erste) Feigenbaumkonstante. Diese beschreibt genau den Faktor, mit dem die Abstände zwischen den Bifuraktionspunkten abnehmen. Dabei ist sie nicht exclusiv für die logistische gleichung, sondern universell für eine ganze Gattung an Gleichungen mit chaotisch-sensitivem Verhalten.
Die Feigenbaumkonstante ist heute auf 1018 Nachkommastellen bekannt und liegt bei etwa 4,669

## Sensitivität

Die Sensitivität eines Systems ist grundlegend die Abhägigkeit dessen von seinen Anfangsparametern. Bei sensitiven Systemen genügen sehr kleine Abweichungen von diesem Anfangszustand um völlig unterschiedliche Verläufe auszulösen.

Ist möglich Regeln zu definieren um den Verlauf zu beschreiben, so spricht man von deterministischem Chaos. Auch bei chaotisch anmutenden Begebenheiten liegen dem deterministischen Chaos immer Regeln zugrunde, mit denen sich das sensitive System beschreiben und voraussagen ließe, würde man es komplett überblicken.

Im Gegenzug dazu gibt es noch vorgänge, welche sich nicht klar durch Regeln beschreiben lassen. Im Fall von quantenmechanischen Prozessen können keine Regelun gefunden werden, da nach heutigem Stand der Wissenschaft quanten sich echt unvorhersehbar verhalten. Wenn ein System nun auf diese quantenmechanischen Prozesse hin sensitiv ist, so lässt sich dessen Verlauf in keinem Fall vorhersagen.

Die bereits beschriebene Logistische Gleichung ist ein Beispiel für eine anfangsbedingungs-sensitive Gleichung.

### Schmetterlingseffekt

Der Schmetterlingseffekt ist wohl der bekanntesten Effekte dem die Sensitivität der Anfangsbedingungen zu Grunde liegt. 
Der Metereologe Edward Norton Lorenz entdeckte das Phänomen 1961, als er Berechnungen zu einem Wettermodell durchführte. Hierbei setzte er verschiedene Wetterparameter un Verbindung und ließ sein Modell mehrere Male berechnen, wobei er jedoch völlig unterschiedliche Ergebnisse erhielt. Dies geschah, weil er bei der erneuten Berechnung Zwischenergebnisse der Ersten verwendete, welche gerundet und weniger genau waren. 



## Julia-Mengen

Definition der ausgefüllten Julia-Menge K des Polynoms f: ℂ → ℂ:
<p style="text-align: center; font-size: 20px;">
<b> K(f) ≔ {z ∈ ℂ | lim<sub>k → ∞</sub> | f<sup>k</sup> (z) | ≠ ∞} </b> </p>  

f<sup>k</sup> ist dabei die k-fache Komposition beziehungsweise Hintereinanderausführung von f. f<sup>2</sup>(x) wäre beispielweise gleich f(f(x)).   
Die ausgefüllte Julia Menge einer Funktion f enthält also alle diejenigen Zahlen z ∈ ℂ, für die die Folge Z<sub>n + 1</sub> = f(Z<sub>n</sub>) mit z als Startwert Z<sub>0</sub> nicht gegen Unendlich strebt, sondern konvergiert.  
Sie kann analog auch über anderen Körpern, beispielsweise den reellen Zahlen, definiert werden. Üblicherweise betrachtet man aber die komplexen Zahlen und wir beschränken uns im Nachfolgenden auch auf diese.
Die Julia Menge ist definiert als Rand der ausgefüllten Julia Menge, oft wird aber auch die gesamte Menge so bezeichnet.

Für viele Funktionen lässt sich vorhersagen, wie sich diese verhalten, wenn man sie wiederholt auf einen Startwert anwendet. Beispielsweise konvergiert f(x) = x / 2 unabhängig vom Startwert gegen 0, f(x) = √x konvergiert für x > 0 gegen 1.  
f(x) = x<sup>2</sup> läuft für alle x mit |x| > 1 gegen +∞, für solche mit 0 < |x| < 1 gegen 0. Besonders sind dabei 0 und 1: Siese sind sogenannte Fixpunkte der Funktion, für die gilt: f(x) = x.
Besonders interessant ist die Julia Menge jedoch für Funktionen, bei denen dieses Verhalten nicht so leicht vorhersehbar ist. Ein populäres Beispiel, auf das die Julia Menge oft angewandt wird, ist die komplexe Funktion *Z<sub>n + 1</sub> = Z<sub>n</sub><sup>2</sup> + c*. Z<sub>n</sub> ist die Laufvariable, c ein beliebig gewählter, komplexer Parameter. 

### Visualisierung

Für komplexe Funktionen kann man die Julia-Menge auf der Gaußschen Zahlenebene darstellen, was für "chaotische" Funktionen wie die obige zu optisch ansprechenden Fraktalen führt, die ihren Teil zur Popularität und Bekanntheit des Problems beigetragen haben dürften. Zahlen, die Element der Julia-Menge sind, werden dabei farbig vom Rest der Zahlenebene abgesetzt. Zusätzlich kann man die nicht zur Julia Menge gehörigen Pixel farblich danach abstufen, wieviele Iterationen nötig waren, um ihre Divergenz festzustellen.

### Implementierung & Performance

Sage stellt eine Methode bereit, um Julia-Mengen darstellen zu können: julia_plot().  
Wie in der Aufgabenstellung gefordert, haben wir aber exemplarisch eine eigene Implementierung in Sage realisiert.  
Zur Darstellung wird die PIL Bibliothek verwendet. Zum einen ist sie wesentlich schneller als beispielsweise eine Visualisierung mittels Sage's plot() und zum anderen erhalten wir so ein Bitmap-Bild, das wir sowohl im Notebook anzeigen als auch für beispielsweise unsere Präsentationen als .PNG exportieren können.  

Um die Performance zu verbessern, nutzen wir Multiprocessing, d.h. wir verteilen die Rechenlast auf verschiedene Prozesse und profitieren somit von modernen Mehrkernprozessoren. Normalerweise nutzt man dazu keine Prozesse, die keinen gemeinsamen Heap haben und als schwere Objekte viel Zeit zur Erstellung in Anspruch nehmen, sondern Threads. Allerdings führten diese in einem Test zu keiner Lastverteilung auf mehrere Kerne, was vermutlich Sage geschuldet ist.  

Nutzt man nun beispielsweise 4 Prozesse, von denen jeder ein Viertel des Bildes errechnet, viertelt das aber noch lange nicht die Laufzeit: Bereiche mit vielen konvergenten Pixeln benötigen wesentlich mehr Iterationen und somit Rechenzeit als Bereiche, in denen bei vielen Pixeln schon nach einer oder zwei Iterationen die Divergenz feststeht, die Prozesse werden also zeitlich weit versetzt fertig. Daher lässt man jeden Prozess an einem Job nur eine gewisse Anzahl Reihen (ROWS_PER_JOB) rechnen und gibt den Job danach an den Hauptprozess zurück. Sind noch weitere Reihen zu bearbeiten, wird der Job entsprechend aktualisiert und an den Rechenprozess zurückgeschickt. So sind alle Prozesse bis zum Schluss gleichmäßig ausgelastet.

Bilder in 1000x1000px Auflösung sind mit oben genannten Optimierungen in wenigen Sekunden machbar.    
Die Rechenzeit für Bilder in annähernd doppelter 4K-Auflösung (4000x4000px) schwankt je nach Julia-Menge, Iterationen, Rechner und interessanterweise vor allem Betriebssystem. Auf verschiedenen Testsystemen benötigte die Julia Menge mit c = -0,7823 + 0,109i, 100 Iterationen, 4000x4000px Auflösung jeweils rund 70 Sekunden unter Ubuntu 20.04, aber fast 6 mal so lang (um 7 Minuten) unter Windows 10. Eventuell ist dies der schlechteren Optimierung der zu Grunde liegenden Bibliotheken geschuldet.  


In [25]:
# MULTI-CORE & PIL BITMAP
# BENÖTIGT MINDESTENS PYTHON 3.7
import multiprocessing
from sage.repl.image import Image
from IPython.display import clear_output
from dataclasses import dataclass
from typing import List, Tuple

NUMBER_OF_PROCESSES = 4  # Anzahl der Prozesse, die zum Berechnen genutzt werden
ROWS_PER_JOB = 100       # Anzahl an Pixelreihen, die pro Job berechnet werden.


class Colorizer:
    """ Stellt verschiedene Farbgebungen für Julia- und Mandelbrot-Mengen bereit """

    @staticmethod
    def fade(fades, result):
        """ 
        Berechnet einen Farbverlauf über die gegebenen Fades.

        Parameters:
            fades (List): Liste von 2er-Tupeln (index, (R, G, B, A)), wobei die RGBA Farbe im Fade genau dann erreicht wird, wenn result == index
            result   (int): Wieviele Iterationen für dieses Pixel durchgeführt wurden
            iter_max (int): Wieviele Iterationen für ein konvergentes Pixel maximal durchgeführt wurden
        """
        for i in range(len(fades)):
            if result <= fades[i][0]:
                progress = (result - fades[i-1][0]) / (fades[i][0] - fades[i-1][0])
                return fades[0][1] if i == 0 else tuple(fades[i-1][1][t] + (fades[i][1][t] - fades[i-1][1][t]) * progress for t in range(4))
    
    @staticmethod
    def get_fades(iter_max):
        """ Gibt eine Übersicht über mögliche fades zurück """
        return {
            'Rot':           [(0,(255, 255, 255, 255)), (iter_max / 10,(255, 200, 200, 255)),                                      (iter_max - 1,(100, 0,   0,   255)), (iter_max,(0,   0,   0,   255))],
            'Grün':          [(0,(255, 255, 255, 255)), (iter_max / 10,(200, 255, 200, 255)),                                      (iter_max - 1,(0,   100, 0,   255)), (iter_max,(0,   0,   0,   255))],
            'Blau':          [(0,(255, 255, 255, 255)), (iter_max / 10,(200, 200, 255, 255)),                                      (iter_max - 1,(0,   0,   100, 255)), (iter_max,(0,   0,   0,   255))],
            'Schatten':      [(0,(255, 255, 255, 255)), (iter_max / 10,(200, 200, 200, 255)),                                      (iter_max - 1,(0,   0,   0,   255)), (iter_max,(255, 255, 255, 255))],
            'Doppler':       [(0,(255, 255, 255, 255)), (iter_max / 10,(0,   0,   100, 255)), (iter_max / 2,(100, 0,   100, 255)), (iter_max - 1,(0,   200, 255, 255)), (iter_max,(255, 255, 255, 255))],
            'Gamma Doppler': [(0,(255, 255, 255, 255)), (iter_max / 10,(0,   200, 0,  255)),  (iter_max / 2,(0,   0,   50,  255)), (iter_max - 1,(0,   200, 255, 255)), (iter_max,(255, 255, 255, 255))],
            'Fade':          [(0,(255, 255, 255, 255)), (iter_max / 10,(255, 255, 0,   255)), (iter_max / 2,(200, 50,  255, 255)), (iter_max - 1,(80,  0,   150, 255)), (iter_max,(255, 255, 255, 255))],
            'Marble Fade':   [(0,(255, 255, 255, 255)), (iter_max / 10,(255, 255, 0,   255)), (iter_max / 2,(0,   0,   255, 255)), (iter_max - 1,(255, 0,   0,   255)), (iter_max,(255, 255, 255, 255))],
            'Random':        [(0,(255, 255, 255, 255))] + [(iter_max * i / 10, (randint(0, 255), randint(0, 255), randint(0, 255), 255)) for i in range(1, 10)] + [(iter_max,(255, 255, 255, 255))]
        }
    
    @staticmethod
    def get_keys():
        """ Gibt eine Liste mit gültigen Keys für das get_fades() dict zurück. 100 ist willkürlich gewählt. """
        return [*Colorizer.get_fades(100)]

    
@dataclass
class RenderJob:
    """ 
    Stellt Objekte zur Kommunikation mit den RenderProcesses bereit
    
    RenderJobs werden zur Kommunikation zwischen dem Hauptprogramm und den RenderProcesses eingesetzt.
    Sie werden mit den Auftragseckdaten erstellt und an einen RenderProcess geschickt, welcher den Job nach Bearbeitung wieder zurücksendet.
    Die errechneten Daten wurden in das Image-Objekt self.image des bearbeiteten Jobs geschrieben.
    Der Job kann danach angepasst und wieder vergeben werden, so dass am gleichen Bild weitergearbeitet werden kann.    
    
    Attributes:
        DIV_LIMIT (int): Bei Überschreitung dieses Z-Wertes wird ein Pixel als divergent betrachtet.
        PX_HEIGHT (float): Gibt die Höhe eines Pixels in der gaußschen Zahlenebene an.
        PX_WIDTH (float): Gibt die Breite eines Pixels in der gaußschen Zahlenebene an.
        image (Image): Bild, auf dem die errechneten Pixel eingetragen werden

    Parameters:
        x_min (int): Der kleinste x-Wert (reeller Anteil), der auf der gaußschen Zahlenebene dargestellt wird
        x_max (int): Der größte x-Wert (reeller Anteil), der auf der gaußschen Zahlenebene dargestellt wird
        y_min (int): Der kleinste y-Wert (imaginärer Anteil), der auf der gaußschen Zahlenebene dargestellt wird
        y_max (int): Der größte y-Wert (imaginärer Anteil), der auf der gaußschen Zahlenebene dargestellt wird
        res_x (int): Horizontale Auflösung des erstellen Bildes. Seitenverhältnis sollte mit Ausschnitt der gaußscher Zahlenebene übereinstimmen!
        res_y (int): Vertikale Auflösung des erstellen Bildes. Seitenverhältnis sollte mit Ausschnitt der gaußscher Zahlenebene übereinstimmen!
        min_row (int): Die oberste Pixelreihe, die mit diesem Auftrag berechnet werden soll (inklusiv) ((0,0) ist die obere linke Ecke bei PIL)
        max_row (int): Die unterste Pixelreihe, die mit diesem Auftrag berechnet werden soll (exklusiv) ((0,0) ist die obere linke Ecke bei PIL)
        iterations (int): Maximale Iterationstiefe, ab der ein Pixel als konvergent betrachtet wird
        julia (boolean): True falls eine Julia-Menge berechnet werden soll. False falls die Mandelbrotmenge berechnet werden soll ()
        c (complex): Komplexer konstanter Parameter c (Julia-Menge) bzw z-Startwert (Mandelbrot-Menge)
        fade (List): Eine Element aus Colorizer.get_fades() bzw. eine Liste von 2er Tupeln
    """
    x_min: int
    x_max: int
    y_min: int
    y_max: int
    res_x: int
    res_y: int
    min_row: int
    max_row: int
    iter_max: int
    julia: bool
    c: complex
    fade: List[Tuple[int, Tuple[int, int, int, int]]]
    div_limit: int = 2
        
    def __post_init__(self): 
        """ Wird nach __init__ in dataclasses aufgerufen. Berechnet Pixelgröße (abgebildet auf gaußsche Zahlenebene) und erstellt image."""
        self.px_height = (self.y_max - self.y_min) / self.res_y
        self.px_width = (self.x_max - self.x_min) / self.res_x
        self.image = Image('RGBA', (self.res_y, self.res_x), color=(0, 0, 0, 0))

    def f(self, z, c):
        """ Polynom f(z) über den komplexen Zahlen, für das die Julia- oder Mandelbrot-Menge berechnet wird. """
        return z * z + c


class RenderProcess(multiprocessing.Process):
    """
    Ein Prozess, der Berechnungen für die Julia- oder Mandelbrotmenge übernimmt
    
    Durch Nutzung (und adäquate Synchronisation) dieser Prozesse kann die Rechenlast auf mehrere Prozessorkerne verteilt werden.
    Attributes: 
        job_queue (Queue): Queue, aus der RenderJobs entnommen und bearbeitet werden
        result_queue (Queue): Queue, in die abgeschlossene RenderJobs gelegt werden
    """
    def __init__(self, job_queue, result_queue):
        """ Erstellt einen neuen RenderProcess mit Parametern äquivalent zu den Klassen-Attributen (siehe RenderProcess) """
        multiprocessing.Process.__init__(self)
        self.job_queue = job_queue
        self.result_queue = result_queue

    def run(self):
        """ Main-Loop des Prozesses. Wartet auf (in der job_queue) eingehene RenderJobs, bearbeitet diese und legt sie in der result_queue wieder ab. """
        while True:
            job = self.job_queue.get()
            pixels = job.image.pixels()
            for row in range(job.min_row, job.max_row):
                self.render_row(job, pixels, row)
            self.result_queue.put(job)

    def render_row(self, job, pixels, row):
        """
        Berechnet eine Pixel-Reihe eines Bildes.
        
        Parameters:
            job (RenderJob): Der RenderJob, der ausgeführt werden soll
            pixels (PixelAccess): PixelAccess Objekt des aktuellen Auftrags (in dieses wird das Ergebnis geschrieben)
            row (int): Pixelreihe, die berechnet werden soll
        """
        # Diskretisierung der gaußschen Zahlenebene - imaginärer Anteil (bleibt pro Reihe gleich)
        img = job.y_max - (row + 1 / 2) * job.px_height
        for col in range(job.res_x):
            # Diskretisierung der gaußschen Zahlenebene - reeller Anteil
            rat = job.x_min + (col + 1 / 2) * job.px_width
            z = complex(rat, img) if job.julia else job.c
            c = job.c if job.julia else complex(rat, img)
            for result in range(job.iter_max + 1):
                # auf Divergenz prüfen. Wenn noch nicht divergent: f(z) anwenden
                if abs(z) > job.div_limit:
                    break
                z = job.f(z, c)
            # Pixel speichern
            pixels[int(col), int(row)] = Colorizer.fade(job.fade, result)


def mc_pil_plot(x, y, res_x, res_y, iterations, julia, c, fade):
    """
    Plottet interaktiv Julia- oder Mandelbrot-Mengen.
    
    Methode nutzt NUMBER_OF_PROCESSES (globale Variable) RenderProcesses gleichzeitig zur Berechnung
    Benötigt die globalen Variablen ROWS_PER_JOB und NUMBER_OF_PROCESSES
    
    Parameters:
        x (int, int): Reeller Bereich der gaußschen Zahlenebene, der dargestellt wird (x_min, x_max)
        y (int, int): Imaginärer Bereich der gaußschen Zahlenebene, der dargestellt wird (x_min, x_max)
        res_x (int): Horizontale Auflösung des generierten Bildes. Seitenverhältnis sollte zum Ausschnitt der gaußschen Zahlenebene passen!
        res_y (int): Vertikale Auflösung des generierten Bildes. Seitenverhältnis sollte zum Ausschnitt der gaußschen Zahlenebene passen!
        iterations (int): Maximale Iterationszahl, ab der ein Pixel als konvergent betrachtet wird
        julia (boolean): Wenn True wird eine Juliamenge berechnet, sonst eine Mandelbrotmenge
        c (complex): Komplexer konstanter Parameter c (Julia-Menge) bzw z-Startwert (Mandelbrot-Menge)
        fade (List): Ein Element aus Colorizer.get_fades() bzw. eine Liste von 2er Tupeln
    """
    # Queues zur Kommunikation mit den Prozessen erstellen
    result_queue = multiprocessing.Queue()
    job_queue = multiprocessing.Queue()
    
    # RenderProzesse erstellen und starten
    processes = []
    for i in range(NUMBER_OF_PROCESSES):
        process = RenderProcess(job_queue, result_queue)
        processes.append(process)
        process.start()

    # Jobs erstellen und an die Prozesse vergeben.
    # Jeder Job berechnet maximal ROWS_PER_JOB (globale Variable) Pixelreihen auf einmal.
    # Die Jobs werden nach Abschluss der Berechnung für die nächsten Reihen wiederverwendet, um den Merge-Aufwand am Ende gering zu halten
    rows_done = 0
    jobs_created = 0
    print("Fortschritt: 0%")
    while rows_done < res_y:
        if jobs_created < NUMBER_OF_PROCESSES:        # Noch weniger als NUMBER_OF_PROCESSES Jobs erstellt (für jeden Prozess einen) -> Neuen Job erstellen
            job = RenderJob(x[0], x[1], y[0], y[1], res_x, res_y, 0, 0, iterations, julia, c, fade)
            jobs_created += 1
        else:                                         # Bereits genug Jobs erstellt -> Auf den nächsten fertigen Job warten und diesen wiederverwenden
            job = result_queue.get()
            clear_output(wait=True)
            print(f"Fortschritt: {int(100 * rows_done / res_y)}%")
        job.min_row, job.max_row = rows_done, rows_done + min(res_y - rows_done, ROWS_PER_JOB)
        rows_done = job.max_row
        job_queue.put(job)

    # Ergebnisse der Prozesse abwarten und auf ein einzelnes Bild mergen
    image = Image('RGBA', (res_y, res_x), color=(0, 0, 0, 0))
    for i in range(jobs_created):
        received = result_queue.get().image.pil
        image.pil.paste(received, (0, 0), received)
    clear_output(wait=True)
    show(image)

    # Prozesse beenden
    for process in processes:
        process.terminate()
        
    return image
    
@interact
def mc_pil_julia_plot(auto_update=False,
                      x=range_slider(-2, 2, default=(-2, 2), step_size=0.1),
                      y=range_slider(-2, 2, default=(-2, 2), step_size=0.1),
                      res_x=slider(400, 4000, default=1000, step_size=100),
                      res_y=slider(400, 4000, default=1000, step_size=100),
                      iterations=slider(10, 10000, default=100, step_size=10, label="Iterationen"),
                      c=input_box("-0,7823 + 0,109i", "c", str),
                      fade=selector(Colorizer.get_keys(), label="Färbung"),
                      save_image=checkbox(False, label="Bild speichern"),
                      image_path=input_box("/home/cedric/Schreibtisch/", "Speicherpfad", str)):
    """
    Plottet interaktiv Julia-Mengen.
    
    Optional kann die Julia Menge als PNG auf dem Rechner abgelegt werden.
    
    Parameters:
        x (int, int): Reeller Bereich der gaußschen Zahlenebene, der dargestellt wird (x_min, x_max)
        y (int, int): Imaginärer Bereich der gaußschen Zahlenebene, der dargestellt wird (x_min, x_max)
        res_x (int): Horizontale Auflösung des generierten Bildes. Seitenverhältnis sollte zum Ausschnitt der gaußschen Zahlenebene passen!
        res_y (int): Vertikale Auflösung des generierten Bildes. Seitenverhältnis sollte zum Ausschnitt der gaußschen Zahlenebene passen!
        iterations (int): Maximale Iterationszahl, ab der ein Pixel als konvergent betrachtet wird
        c (String): Konstanter komplexer Parameter c. Wird in der Funktion nach complex geparsed, sollte Form 0,02 + 0,54i haben.
        fade (String): Key für Colorizer.get_fades(). Mögliche Keys können Colorizer.get_keys() entnommen werden
        save_image (boolean): True falls versucht werden soll, das Bild zu speichern. False sonst.
        image_path (String): Pfad auf dem Rechner, an dem das Bild gespeichert wird (ohne Dateinamen). Muss auf einen Separator enden (/ oder \\ je nach OS)
    """

    # Komplexen Parameter c in eine komplexe Zahl parsen. Falls nicht möglich: Fehler werfen
    try:
        c = complex(c.replace(" ", "").replace("i", "j").replace(",", "."))
    except ValueError:
        print("Ungültiger komplexer Parameter c! Bitte in der Form 0,23 + 0,43i angeben!")
        return
    
    # Bild berechnen und ausgeben
    image = mc_pil_plot(x, y, res_x, res_y, iterations, true, c, Colorizer.get_fades(iterations)[fade])

    # Speichern des Bildes falls gewünscht
    if save_image:
        image_name = f"Julia Menge (C = {c}, {res_x}x{res_y}, {iterations} Iterationen).png"
        image.save(image_path + image_name)

Manual interactive function <function mc_pil_julia_plot at 0x6fe1c083c80> with 9 widgets
  x: TransformFloatRa…


## Mandelbrotmenge

Aehnlich zur Julia-Menge wird die Mandelbrotmenge berechnet. Dabei haben die Variablen der Formel jedoch eine andere Bedeutung:
Z0 beginnt bei jeder komplexen Zahl mit 0.
c ist eine Zahl auf der komplexen Ebene. <br />
z ist das Ergebnis der letzten rekursion auf dieser komplexen Zahl. <br />
n ist die Genauigkeit.

Die Mandelbrotmenge ist der Bereich, bei welchem Z nach n iterationen nicht den Radius g ueberschreitet und somit zu 0 konvergiert.

In [27]:
@interact
def mc_pil_mandelbrot_plot(auto_update=False,
                      x=range_slider(-2, 2, default=(-2, 2), step_size=0.1),
                      y=range_slider(-2, 2, default=(-2, 2), step_size=0.1),
                      res_x=slider(400, 4000, default=1000, step_size=100),
                      res_y=slider(400, 4000, default=1000, step_size=100),
                      iterations=slider(100, 10000, default=100, step_size=100, label="Iterationen"),
                      z=input_box("0 + 0i", "z<sub>0</sub>", str),
                      fade=selector(Colorizer.get_keys(), label="Färbung"),
                      save_image=checkbox(False, label="Bild speichern"),
                      image_path=input_box("/home/cedric/Schreibtisch/", "Speicherpfad", str)):
    """
    Plottet interaktiv Mandelbrot-Mengen.
    
    Optional kann die Mandelbrot Menge als PNG auf dem Rechner abgelegt werden.
    
    Parameters:
        x (int, int): Reeller Bereich der gaußschen Zahlenebene, der dargestellt wird (x_min, x_max)
        y (int, int): Imaginärer Bereich der gaußschen Zahlenebene, der dargestellt wird (x_min, x_max)
        res_x (int): Horizontale Auflösung des generierten Bildes. Seitenverhältnis sollte zum Ausschnitt der gaußschen Zahlenebene passen!
        res_y (int): Vertikale Auflösung des generierten Bildes. Seitenverhältnis sollte zum Ausschnitt der gaußschen Zahlenebene passen!
        iterations (int): Maximale Iterationszahl, ab der ein Pixel als konvergent betrachtet wird
        z (String): Z-Startwert (für die Standard-Mandelbrotmenge = 0). Wird in der Funktion nach complex geparsed, sollte Form 0,02 + 0,54i haben.
        fade (String): Key für Colorizer.get_fades(). Mögliche Keys können Colorizer.get_keys() entnommen werden
        save_image (boolean): True falls versucht werden soll, das Bild zu speichern. False sonst.
        image_path (String): Pfad auf dem Rechner, an dem das Bild gespeichert wird (ohne Dateinamen). Muss auf einen Separator enden (/ oder \\ je nach OS, Doppelbackslash da sonst invalid escape sequence)
    """

    # Startwert z in eine komplexe Zahl parsen. Falls nicht möglich: Fehler werfen
    try:
        z = complex(z.replace(" ", "").replace("i", "j").replace(",", "."))
    except ValueError:
        print("Ungültiger komplexer Parameter z! Bitte in der Form 0,23 + 0,43i angeben!")
        return
    
    # Bild berechnen und ausgeben
    image = mc_pil_plot(x, y, res_x, res_y, iterations, false, z, Colorizer.get_fades(iterations)[fade])

    # Speichern des Bildes falls gewünscht
    if save_image:
        image_name = f"Mandelbrot Menge (Z0 = {z}, {res_x}x{res_y}, {iterations} Iterationen).png"
        image.save(image_path + image_name)

Manual interactive function <function mc_pil_mandelbrot_plot at 0x6fadab72488> with 9 widgets
  x: TransformFl…

## Apfelmännchen und Selbstähnlichkeit

Apfelmaenchen ist eine visuelle Darstellung der Mandelbrotmenge, wobei die X Achse auf dem Koordinatensystem der reele und die Y Achse der imaginaere Teil der Komplexen Zahl ist.

Zeichnen:
Um die Menge zu zeichnen muss man ueber jeden Pixel iterieren und auf diesen die gegebene Formel n mal anwenden. Falls der Wert ueber den Radius g (Bei der klassischen Mandelbrotmenge 2) schreitet, wird anhand der benoetigten Iterationen die Farbe festgestellt. Wenn dieser Schwellwert nach n iterationen nicht ueberschritten wird, wird das Pixel schwarz gefaerbt.

Mit anpassen der Formel, beispielsweise zu Z^3 + c, ergeben sich andere Fraktale mit eigenen Eigenschaften.

# Quellen
Logistische Gleichung:

- Die logistische Gleichung - ein Weg ins Chaos (Andreas Schmid, 2009, http://www.fraktalwelt.de/systeme/as_weg_ins_chaos.pdf)
- Modellierung natürlicher Prozesse und Optimierungsstrategien (Jens Kortus, Theoretische Physik TU Freiberg (https://tu-freiberg.de/sites/default/files/media/institut-fuer-theoretische-physik-10451/Lehre/Modelierung/modnatproz.pdf)
- Populationsmodelle (Philipp Jansche, 2008/09, https://www.mathi.uni-heidelberg.de/~thaeter/mathmod08/AusarbeitungPopulationsmodelle.pdf)
- Die logistische Gleichung als ein Beispiel für chaotische Prozesse in der Physik (Nils Grzech, 2012, https://docplayer.org/18693722-Die-logistische-gleichung-als-ein-beispiel-fuer-chaotische-prozesse-in-der-physik.html)

Feigenbaum-Diagramm:

- Feigenbaum-Konstante https://physik.cosmos-indirekt.de/Physik-Schule/Feigenbaum-Konstante, https://www.heise.de/newsticker/meldung/Zahlen-bitte-Die-Feigenbaum-Konstante-beschreibt-Ordnung-im-Chaos-4141733.html
- Diagramm erzeugen http://walter.bislins.ch/blog/index.asp?page=Feigenbaum%2DDiagramm+erzeugen+und+analysieren

Sensitivität:

- Chaosforschung https://physik.cosmos-indirekt.de/Physik-Schule/Chaosforschung
- Schmetterlingseffekt:
    - https://www.br.de/wissen/edward-lorenz-meteorologe-schmetterlingseffekt-chaostheorie-100.html 
    - https://physik.cosmos-indirekt.de/Physik-Schule/Schmetterlingseffekt
    
Julia-Mengen:

- Möglichkeiten der Darstellung von Julia-Mengen und Apfelmännchen (Lukas von Stumberg, 2011, https://www.hans-riegel-fachpreise.com/fileadmin/hans-riegel-fachpreise/Module/ausgezeichnete-arbeiten/hans-riegel-fachpreise-seminararbeit-vwa-2012-von-stumberg.pdf)
- Fraktale Geometrie: Julia Mengen (Gunnar Völkel, 2007, http://www.mathematik.uni-ulm.de/stochastik/lehre/ws06_07/seminar_fraktale/ausarbeitung_voelkel.pdf und http://www.mathematik.uni-ulm.de/stochastik/lehre/ws06_07/seminar_fraktale/voelkel.pdf)

    
The dark side of the Mandelbrot set / Mathologer: <br />
https://www.youtube.com/watch?v=9gk_8mQuerg

Die Mandelbrotmenge / Uni-Leipzig: <br />
https://www.informatik.uni-leipzig.de/~meiler/Schuelerseiten.dir/DPlotzki/html/mndlbrt.htm

# Kollaboration
Da unser Thema sechs Unterthemen und unser Team drei Teilnehmer umfasst, hat sich jeder Teilnehmer mit primär zwei Themen beschäftigt.  
Natürlich haben die sechs Themen viele Schnittpunkte gemein und die Trennlinie ist daher nicht scharf zu ziehen, aber grob haben wir die Themen folgendermaßen aufgeteilt:
- Michael Rudyj: Feigenbaum-Diagramm & Sensitivität  
- Cedric Holzer: Logistische Gleichung & Julia-Mengen  
- Aron Winter: Mandelbrotmenge & Apfelmännchen und Selbstähnlichkeit  